# Exercise 2

### Part 1. Implementing a merkle tree

The general algorithm, I think, is to just split data blocks into adjacent pairs, hash them separately and create a hashpointer from each to a new node in the level above labeled with the hash of the concatenation of the two original hashes, then repeat until you don't have pairs on any levels without parents. 

In [1]:
from hashlib import sha256
import math

In [2]:
class Node():
    def __init__(self, data):
        self.data = data
        self.leftChild = None
        self.rightChild = None
        self.parent = None

In [3]:
class MerkleTree():
    def __init__(self, data, **kwargs):
        self.data = data
        self.testFlag = kwargs['testFlag'] if 'testFlag' in kwargs else False
        self.createTree()
        
    def hashFunction(self, datainput):
        if self.testFlag:
            return 'H({})'.format(datainput)
        else:
            sha_compute = sha256()
            sha_compute.update(datainput)
            return sha_compute.digest()

    def createTree(self):
        self.sorted_root = None
        self.root = None
        self.sorted_nodes = []
        self.nodes = []
        
        def createLeaves():
            # TODO: I am assuming even number of input blocks, but generally not the case
            firstLevel, firstLevel_sorted = [], []
            for block in data:
                newNode = Node(self.hashFunction(block))
                firstLevel.append(newNode)
            self.nodes.append(firstLevel)
            for block in sorted(data):
                newNode = Node(self.hashFunction(block))
                firstLevel_sorted.append(newNode)
            self.sorted_nodes.append(firstLevel_sorted)
    
        def createUnsortedTree():
            '''
            Note: this outputs a tree in the same way as createTree(), except we sort the data blocks beforehand. 
            '''
            currLevel = 0
            while len(self.nodes[currLevel]) > 1:
                self.nodes.append([])
                # get pairs of adjacent blocks (1+2, 3+4, ...)
                pairs = [x.data+y.data for (x,y) in list(zip(self.nodes[currLevel][::2],self.nodes[currLevel][1::2]))]
                count = 0
                for pair in pairs:
                    newNode = Node( self.hashFunction(pair) )
                    newNode.leftChild = self.nodes[currLevel][count]
                    newNode.rightChild = self.nodes[currLevel][count+1]
                    self.nodes[currLevel+1].append(newNode)
                    self.nodes[currLevel][count].parent = self.nodes[currLevel+1][-1] 
                    self.nodes[currLevel][count+1].parent = self.nodes[currLevel+1][-1]
                    count += 2
                currLevel += 1
            self.root = self.nodes[-1][0]

        def createSortedTree():
            currLevel = 0
            while len(self.sorted_nodes[currLevel]) > 1:
                self.sorted_nodes.append([])
                # get pairs of adjacent blocks (1+2, 3+4, ...)
                pairs = [x.data+y.data for (x,y) in list(zip(self.sorted_nodes[currLevel][::2],self.sorted_nodes[currLevel][1::2]))]
                count = 0
                for pair in pairs:
                    newNode = Node( self.hashFunction(pair) )
                    newNode.leftChild = self.sorted_nodes[currLevel][count]
                    newNode.rightChild = self.sorted_nodes[currLevel][count+1]
                    self.sorted_nodes[currLevel+1].append(newNode)
                    self.sorted_nodes[currLevel][count].parent = self.sorted_nodes[currLevel+1][-1] 
                    self.sorted_nodes[currLevel][count+1].parent = self.sorted_nodes[currLevel+1][-1]
                    count += 2
                currLevel += 1
            self.sorted_root = self.sorted_nodes[currLevel][0]
        
        createLeaves()
        createUnsortedTree()
        createSortedTree()
    
    def verifyBlock(self, datablock, pos):
        num_blocks = len(self.data)
        level = int(math.log(num_blocks)/math.log(2))
        my_data = datablock
        prevNode = 0
        def getRoot(level, prevNode, minleaf=0, maxleaf=float(num_blocks-1)):
            if level == 0:
                return my_data
            midpoint = minleaf+(maxleaf-minleaf)/2
            if pos >= midpoint:
                minleaf += (maxleaf-minleaf+1)/2
                return (test.nodes[level-1][prevNode*2].data + self.hashFunction(getRoot(level-1,prevNode*2+1,minleaf, maxleaf)))
            elif pos < midpoint:
                maxleaf -= (maxleaf-minleaf+1)/2
                return (self.hashFunction(getRoot(level-1,prevNode*2,minleaf, maxleaf)) + test.nodes[level-1][prevNode*2+1].data)
        
        print('Expected root: {}'.format(self.root.data))
        print('Calculated root: {}'.format(self.hashFunction(getRoot(level, prevNode))))
        return self.root.data == self.hashFunction(getRoot(level, prevNode))

    

### Testing Functionalities

#### First we will demonstrate the usability of the merkle tree class based on this set of data blocks.

In [4]:
data = [b'four score and seven years ago', b"our fathers brought forth",
        b"on this continent", b"a new nation", b"conceived in Liberty", 
        b"and dedicated", b"to the proposition that", b"all men are created equal",
       b"that this nation", b"under God", b"shall have a new birth of freedom",
        b"and that government", b"of the people", b"by the people", b"for the people", 
        b"shall not perish from the earth"]


In [5]:
test = MerkleTree(data)
test.nodes

[[<__main__.Node at 0x21b97934400>,
 [<__main__.Node at 0x21b97934f98>, <__main__.Node at 0x21b97934fd0>],

As shown, the data is used to create a merkle tree. The set of nodes created are stored in the MerkleTree object as an attribute called nodes, representing the "server knowledge", which grants validation users quick access to nodes. The nodes are stored in the form of an array, where the test.nodes[0] represents the lowest level (a.k.a hash of data blocks), and test.nodes[-1] represents the highest level (a.k.a root of the merkle tree).

In this case, access to node information is constant time O(1), but takes log space O(logn), where n is the number of nodes. (Note: set of nodes does not include data blocks, but it DOES include the hash of the data blocks! This is the whole point of a merkle tree) 

#### Given a set of data blocks, we will check if certain data blocks are in the merkle tree at certain positions

For proof of concept, we will not using an actual hash function for this demo, but instead be visualizing the hashing operation as H(). In the next cell, we will perform the same operations with an hash function. 

In [6]:
data = [b'0',b'1',b'2',b'3']
test = MerkleTree(data, testFlag=True)
print( test.verifyBlock(b'3', 2) )
print()
print( test.verifyBlock(b'4', 1) )
print()
print( test.verifyBlock(b'1', 1) )


Expected root: H(H(H(b'0')H(b'1'))H(H(b'2')H(b'3')))
Calculated root: H(H(H(b'0')H(b'1'))H(H(b'3')H(b'3')))
False

Expected root: H(H(H(b'0')H(b'1'))H(H(b'2')H(b'3')))
Calculated root: H(H(H(b'0')H(b'4'))H(H(b'2')H(b'3')))
False

Expected root: H(H(H(b'0')H(b'1'))H(H(b'2')H(b'3')))
Calculated root: H(H(H(b'0')H(b'1'))H(H(b'2')H(b'3')))
True


In [7]:
data = [b'0',b'1',b'2',b'3']
test = MerkleTree(data, testFlag=False)
print( test.verifyBlock(b'3', 2) )
print()
print( test.verifyBlock(b'4', 1) )
print()
print( test.verifyBlock(b'1', 1) )


Expected root: b'\xc4x\xfe\xad\x0c\x89\xb7\x95@c\x8f\x84L\x88\x19\xd9\xa4(\x17c\xaf\x92r\xc7\xf3\x96\x87v\xb6\x05#E'
Calculated root: b'\xc4\xddG\xac)_u\r\xbe#\x89g\xc5\x83\x18c\xa0\xea\xfe\xa9\x92\x1e\xd4V\x1c\xdc\x0b\tv9\xdd\xcf'
False

Expected root: b'\xc4x\xfe\xad\x0c\x89\xb7\x95@c\x8f\x84L\x88\x19\xd9\xa4(\x17c\xaf\x92r\xc7\xf3\x96\x87v\xb6\x05#E'
Calculated root: b'm\x11\xa96\xc7\xd0\xa4\xe8\xed\x9cz\x98\x8c\xd3 \xd7j\xc6\x18M\xbd\xbf=\x0fZ\x95\x8e[}wI\x00'
False

Expected root: b'\xc4x\xfe\xad\x0c\x89\xb7\x95@c\x8f\x84L\x88\x19\xd9\xa4(\x17c\xaf\x92r\xc7\xf3\x96\x87v\xb6\x05#E'
Calculated root: b'\xc4x\xfe\xad\x0c\x89\xb7\x95@c\x8f\x84L\x88\x19\xd9\xa4(\x17c\xaf\x92r\xc7\xf3\x96\x87v\xb6\x05#E'
True
